# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 3 2023 1:38PM,258647,16,SSF_C1W10,Sartorius Stedim Filters Inc.,Released
1,Mar 3 2023 1:25PM,258646,19,ADV80013162,"AdvaGen Pharma, Ltd",Released
2,Mar 3 2023 1:25PM,258646,19,ADV80013163,"AdvaGen Pharma, Ltd",Released
3,Mar 3 2023 1:25PM,258646,19,ADV80013164,"AdvaGen Pharma, Ltd",Executing
4,Mar 3 2023 1:25PM,258646,19,ADV80013166,"AdvaGen Pharma, Ltd",Executing
5,Mar 3 2023 1:25PM,258646,19,ADV80013167,"AdvaGen Pharma, Ltd",Released
6,Mar 3 2023 1:15PM,258645,16,9228096,Sartorius Bioprocess Solutions,Released
7,Mar 3 2023 1:14PM,258644,16,9228086,Sartorius Bioprocess Solutions,Released
8,Mar 3 2023 12:36PM,258641,18,INNOVA-9227567,Innova Softgel LLC,Executing
9,Mar 3 2023 12:32PM,258640,10,CTF0011667,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,258644,Released,1
20,258645,Released,1
21,258646,Executing,2
22,258646,Released,3
23,258647,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258641,1.0,NaN
258644,NaN,1.0
258645,NaN,1.0
258646,2.0,3.0
258647,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258578,1.0,0.0
258599,1.0,0.0
258612,2.0,12.0
258617,0.0,1.0
258618,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258578,1,0
258599,1,0
258612,2,12
258617,0,1
258618,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258578,1,0
1,258599,1,0
2,258612,2,12
3,258617,0,1
4,258618,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258578,1,
1,258599,1,
2,258612,2,12
3,258617,,1
4,258618,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 3 2023 1:38PM,258647,16,Sartorius Stedim Filters Inc.
1,Mar 3 2023 1:25PM,258646,19,"AdvaGen Pharma, Ltd"
6,Mar 3 2023 1:15PM,258645,16,Sartorius Bioprocess Solutions
7,Mar 3 2023 1:14PM,258644,16,Sartorius Bioprocess Solutions
8,Mar 3 2023 12:36PM,258641,18,Innova Softgel LLC
9,Mar 3 2023 12:32PM,258640,10,"Citieffe, Inc."
10,Mar 3 2023 12:18PM,258639,10,Eywa Pharma Inc.
23,Mar 3 2023 12:02PM,258637,10,MedStone Pharma LLC
31,Mar 3 2023 11:58AM,258636,10,MedStone Pharma LLC
32,Mar 3 2023 11:43AM,258633,10,"Digital Brands, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 3 2023 1:38PM,258647,16,Sartorius Stedim Filters Inc.,,1
1,Mar 3 2023 1:25PM,258646,19,"AdvaGen Pharma, Ltd",2,3
2,Mar 3 2023 1:15PM,258645,16,Sartorius Bioprocess Solutions,,1
3,Mar 3 2023 1:14PM,258644,16,Sartorius Bioprocess Solutions,,1
4,Mar 3 2023 12:36PM,258641,18,Innova Softgel LLC,1,
5,Mar 3 2023 12:32PM,258640,10,"Citieffe, Inc.",,1
6,Mar 3 2023 12:18PM,258639,10,Eywa Pharma Inc.,,13
7,Mar 3 2023 12:02PM,258637,10,MedStone Pharma LLC,3,5
8,Mar 3 2023 11:58AM,258636,10,MedStone Pharma LLC,,1
9,Mar 3 2023 11:43AM,258633,10,"Digital Brands, LLC",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 1:38PM,258647,16,Sartorius Stedim Filters Inc.,1,
1,Mar 3 2023 1:25PM,258646,19,"AdvaGen Pharma, Ltd",3,2
2,Mar 3 2023 1:15PM,258645,16,Sartorius Bioprocess Solutions,1,
3,Mar 3 2023 1:14PM,258644,16,Sartorius Bioprocess Solutions,1,
4,Mar 3 2023 12:36PM,258641,18,Innova Softgel LLC,,1
5,Mar 3 2023 12:32PM,258640,10,"Citieffe, Inc.",1,
6,Mar 3 2023 12:18PM,258639,10,Eywa Pharma Inc.,13,
7,Mar 3 2023 12:02PM,258637,10,MedStone Pharma LLC,5,3
8,Mar 3 2023 11:58AM,258636,10,MedStone Pharma LLC,1,
9,Mar 3 2023 11:43AM,258633,10,"Digital Brands, LLC",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 1:38PM,258647,16,Sartorius Stedim Filters Inc.,1,
1,Mar 3 2023 1:25PM,258646,19,"AdvaGen Pharma, Ltd",3,2
2,Mar 3 2023 1:15PM,258645,16,Sartorius Bioprocess Solutions,1,
3,Mar 3 2023 1:14PM,258644,16,Sartorius Bioprocess Solutions,1,
4,Mar 3 2023 12:36PM,258641,18,Innova Softgel LLC,,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 1:38PM,258647,16,Sartorius Stedim Filters Inc.,1.0,NaN
1,Mar 3 2023 1:25PM,258646,19,"AdvaGen Pharma, Ltd",3.0,2.0
2,Mar 3 2023 1:15PM,258645,16,Sartorius Bioprocess Solutions,1.0,NaN
3,Mar 3 2023 1:14PM,258644,16,Sartorius Bioprocess Solutions,1.0,NaN
4,Mar 3 2023 12:36PM,258641,18,Innova Softgel LLC,NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 1:38PM,258647,16,Sartorius Stedim Filters Inc.,1.0,0.0
1,Mar 3 2023 1:25PM,258646,19,"AdvaGen Pharma, Ltd",3.0,2.0
2,Mar 3 2023 1:15PM,258645,16,Sartorius Bioprocess Solutions,1.0,0.0
3,Mar 3 2023 1:14PM,258644,16,Sartorius Bioprocess Solutions,1.0,0.0
4,Mar 3 2023 12:36PM,258641,18,Innova Softgel LLC,0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2068997,35.0,16.0
12,517227,0.0,2.0
16,775936,3.0,0.0
18,258641,0.0,1.0
19,1293128,8.0,2.0
20,258627,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2068997,35.0,16.0
1,12,517227,0.0,2.0
2,16,775936,3.0,0.0
3,18,258641,0.0,1.0
4,19,1293128,8.0,2.0
5,20,258627,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,35.0,16.0
1,12,0.0,2.0
2,16,3.0,0.0
3,18,0.0,1.0
4,19,8.0,2.0
5,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,35.0
1,12,Released,0.0
2,16,Released,3.0
3,18,Released,0.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Executing,16.0,2.0,0.0,1.0,2.0,0.0
Released,35.0,0.0,3.0,0.0,8.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Executing,16.0,2.0,0.0,1.0,2.0,0.0
1,Released,35.0,0.0,3.0,0.0,8.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Executing,16.0,2.0,0.0,1.0,2.0,0.0
1,Released,35.0,0.0,3.0,0.0,8.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()